## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")                              
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chokurdakh,RU,70.6333,147.9167,19.99,96,96,8.68,overcast clouds
1,1,Bonavista,CA,48.6499,-53.1147,59.00,88,69,12.30,broken clouds
2,2,Kaitangata,NZ,-46.2817,169.8464,49.93,82,100,13.67,light rain
3,3,Mansfield,GB,53.1333,-1.2000,54.99,88,4,1.99,clear sky
4,4,Ushuaia,AR,-54.8000,-68.3000,47.86,61,75,23.02,moderate rain


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp= float(input("What is the minimum temperature you would like for your trip?"))
max_temp= float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Puerto Ayora,EC,-0.7393,-90.3518,76.96,76,30,11.86,scattered clouds
17,17,Waspan,NI,14.7419,-83.9717,87.48,73,96,4.97,overcast clouds
19,19,Avarua,CK,-21.2078,-159.7750,75.25,73,75,6.91,broken clouds
22,22,Hambantota,LK,6.1241,81.1185,79.14,100,0,4.61,clear sky
26,26,Sawakin,SD,19.1059,37.3321,85.78,60,98,8.68,overcast clouds
28,28,Arraial Do Cabo,BR,-22.9661,-42.0278,85.96,69,20,23.02,few clouds
30,30,Butaritari,KI,3.0707,172.7902,82.36,65,100,17.63,overcast clouds
33,33,Avera,US,33.1940,-82.5271,87.44,41,80,4.54,broken clouds
39,39,Alyangula,AU,-13.8483,136.4192,80.47,78,90,5.75,overcast clouds
43,43,Malpe,IN,13.3500,74.7167,81.70,90,90,2.06,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Puerto Ayora,EC,-0.7393,-90.3518,76.96,76,30,11.86,scattered clouds
17,17,Waspan,NI,14.7419,-83.9717,87.48,73,96,4.97,overcast clouds
19,19,Avarua,CK,-21.2078,-159.7750,75.25,73,75,6.91,broken clouds
22,22,Hambantota,LK,6.1241,81.1185,79.14,100,0,4.61,clear sky
26,26,Sawakin,SD,19.1059,37.3321,85.78,60,98,8.68,overcast clouds


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Puerto Ayora,EC,76.96,scattered clouds,-0.7393,-90.3518,
17,Waspan,NI,87.48,overcast clouds,14.7419,-83.9717,
19,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
22,Hambantota,LK,79.14,clear sky,6.1241,81.1185,
26,Sawakin,SD,85.78,overcast clouds,19.1059,37.3321,
28,Arraial Do Cabo,BR,85.96,few clouds,-22.9661,-42.0278,
30,Butaritari,KI,82.36,overcast clouds,3.0707,172.7902,
33,Avera,US,87.44,broken clouds,33.1940,-82.5271,
39,Alyangula,AU,80.47,overcast clouds,-13.8483,136.4192,
43,Malpe,IN,81.70,overcast clouds,13.3500,74.7167,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c.1 Add the lat and lng to location key for the params dictionary
    params["location"]=f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [37]:
# 6f.1 check if there is any null value in hotel_df
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [43]:
# 6f.2 Pandas recognizes a value as null if it is a np.nan object, prints it as NaN in the DataFrame.
#Pandas does not recognize empty strings (missing hotel name) as null
# To fix this, convert the empty strings to np.nan objects using replace()
# Re-count the null value in hotel name column
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.isnull().sum()

#source: https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             14
dtype: int64

In [44]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna( how='any',
                    subset=['Hotel Name'])
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Puerto Ayora,EC,76.96,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
17,Waspan,NI,87.48,overcast clouds,14.7419,-83.9717,Hostal casa de la rose
19,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn
22,Hambantota,LK,79.14,clear sky,6.1241,81.1185,Bungalow 63
26,Sawakin,SD,85.78,overcast clouds,19.1059,37.3321,فندق الدويم
28,Arraial Do Cabo,BR,85.96,few clouds,-22.9661,-42.0278,Pousada Porto Praia
30,Butaritari,KI,82.36,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
39,Alyangula,AU,80.47,overcast clouds,-13.8483,136.4192,Groote Eylandt Lodge
43,Malpe,IN,81.70,overcast clouds,13.3500,74.7167,Hotel Sri Krishna Residency
47,Nago,JP,81.03,clear sky,26.5881,127.9761,Hotel Yugaf Inn Okinawa


In [45]:
# 8a. Create the output File (CSV)
output_data_file = "..\Vacation_Search\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [53]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer= gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))